In [3]:
import openpyxl
from itertools import islice

# 엑셀파일 열기
file = openpyxl.load_workbook('items.xlsx')

# sheet 가져오기
sett_sheet = file['set']
head_sheet = file['head']
body_sheet = file['body']
gloves_sheet = file['gloves']
belt_sheet = file['belt']
shoes_sheet = file['shoes']

# 장비 list 가져오기
HEAD, BODY, GLOVES, BELT, SHOES = [], [], [], [], []
SETS = dict()

# 아이템 가져오기 (이름, 힘, 민, 생, 지, 타저, 마저, 추공, 부위, 셋트, 활성여부)
for row in islice(head_sheet.values, 1, None):
    if row[10]: # 활성여부
        HEAD.append([x if x else 0 for x in row])
    
for row in islice(body_sheet.values, 1, None):
    if row[10]: # 활성여부
        BODY.append([x if x else 0 for x in row])
    
for row in islice(gloves_sheet.values, 1, None):
    if row[10]: # 활성여부
        GLOVES.append([x if x else 0 for x in row])
    
for row in islice(belt_sheet.values, 1, None):
    if row[10]: # 활성여부    
        BELT.append([x if x else 0 for x in row])
    
for row in islice(shoes_sheet.values, 1, None):
    if row[10]: # 활성여부
        SHOES.append([x if x else 0 for x in row])

# set효과 가져오기
for row in islice(sett_sheet.values, 1, None):
    row = [x if x else 0 for x in row]
    sett, num, HIM, MIN, SAENG, JI, TA, MA = row
    
    if sett in SETS:
        SETS[sett][num] = (HIM, MIN, SAENG, JI, TA, MA)
    else:
        SETS[sett] = dict()
        SETS[sett][num] = (HIM, MIN, SAENG, JI, TA, MA)

In [4]:
# 모든 방어구 조합 생성
from itertools import product
def get_all_comb():
    return product(HEAD, BODY, GLOVES, BELT, SHOES)

In [5]:
# 조합의 스텟 총합 계산
# 활성화된 SET효과도 표시
def get_total_stats(comb):
    HIM, MIN, SAENG, JI, TA, MA, CHU, SETT, NAMES = 0, 0, 0, 0, 0, 0, 0, {}, []
    # 아이템 개별 스탯 적용
    for item in comb:
        name, him, minn, saeng, ji, ta, ma, chu, part, sett, inactive = item
        NAMES.append(name)
        HIM += him
        MIN += minn
        SAENG += saeng
        JI += ji
        TA += ta
        MA += ma
        CHU += chu
        if sett in SETT:
            SETT[sett] += 1
        else:
            SETT[sett] = 1
            
    # 아이템 셋트효과 적용
    for sett in SETT:
        # 수련용 수룡왕 갑투 등은 셋트효과가 없음 => skip
        if sett not in SETS:
            continue

        num = SETT[sett]
        for i in range(1, num+1):
            if i in SETS[sett]:
                him, minn, saeng, ji, ta, ma = SETS[sett][i]
                HIM += him
                MIN += minn
                SAENG += saeng
                JI += ji
                TA += ta
                MA += ma
                
    return (HIM, MIN, SAENG, JI, TA, MA, CHU, NAMES)

In [6]:
# 최종 결과를 출력해주는 함수
def print_result(result):
    stats = ['힘', '민', '생', '지', '타저', '마저', '추공', '아이템']
    header = ''
    for stat in stats:
        header += stat.ljust(4)
    print(header)
    for row in result:
        row_str = ''
        for stat in row:
            row_str += str(stat).rjust(4)
            
        print(row_str)

In [15]:
# stat계산기의 결과를 정렬해주는 함수
def sort_result(result, *stats, order='ASC'):
    STAT = {
        '힘': 0,
        '민': 1,
        '생': 2,
        '지': 3,
        '타저': 4,
        '마저': 5,
        '추공': 6,
    }
    basis = tuple([STAT[x] for x in stats])
    result.sort(key=lambda x: [x[i] for i in basis])
    
    if order=='DESC':
        result.reverse()
    return

In [16]:
# 스텟셋팅
results = []
for comb in get_all_comb():
    stat = list(get_total_stats(comb))
    results.append(stat)

# 힘 민 생 지 타저 마저 추공
results = list(filter(lambda x: x[0] > 1000, results))
sort_result(results, '힘', '지', order='DESC')
print('total', len(results), 'combinations')
print_result(results[:])

total 952 combinations
힘   민   생   지   타저  마저  추공  아이템 
1250   0 525 175 160 100   0['브리트라투구', '황제의갑옷', '황제의장갑', '황제의요대', '황제의신발']
1250   0 575 175 180 115   0['황제의머리띠', '황제의갑옷', '황제의장갑', '황제의요대', '황제의신발']
1250   0 525 150 170  95   0['황제의머리띠', '백호피갑', '황제의장갑', '황제의요대', '황제의신발']
1250   0 550 150 175  95   0['황제의머리띠', '황제의갑옷', '백호손목보호대', '황제의요대', '황제의신발']
1225   0 550 175 170 100   0['황제의머리띠', '황제의갑옷', '황제의장갑', '황제의요대', '파멸자의전투화']
1225   0 525 150 180  90   0['황제의머리띠', '황제의갑옷', '황제의장갑', '황제의요대', '백호의전투화']
1225   0 350 125 145 110   0['브리트라투구', '백호피갑', '백호손목보호대', '황제의요대', '황제의신발']
1225   0 500 125 165 100   0['황제의머리띠', '백호피갑', '백호손목보호대', '황제의요대', '황제의신발']
1205 130 515 140 190  80   0['황제의머리띠', '황제의갑옷', '황제의장갑', '황제의요대', '담덕태황의신발']
1200   0 475 225 180  90   0['황제의머리띠', '황제의갑옷', '황제의장갑', '파멸자의요대', '황제의신발']
1200   0 575 225 180  90   0['황제의머리띠', '황제의갑옷', '황제의장갑', '치우천왕요대', '황제의신발']
1200   0 550 200 165 100   0['백호의머리띠', '황제의갑옷', '황제의장갑', '황제의요대', '황제의신발']
1200   0 550 200 175  90   0['황제의머

In [19]:
### 타저 마저 셋팅
character_hit = int(input('기본 타저'))
character_magic = int(input('기본 마저'))
min_hit = int(input('최소타저'))
min_magic = int(input('최소마저'))
results = []

for comb in get_all_comb():
    stat = list(get_total_stats(comb))
    TA = stat[4] + character_hit # 타저
    MA = stat[5] + character_magic # 마저
    if TA >= min_hit and MA >= min_magic:
        stat[4] = TA # 기본타저 적용
        stat[5] = MA # 기본마저 적용
        results.append(stat)
        
results.sort(key = lambda x: (x[4], x[5]), reverse=True) # (타저,마저) asc
print('total', len(results), 'combinations')
print_result(results[:])

기본 타저70
기본 마저30
최소타저150
최소마저150
total 4709 combinations
힘   민   생   지   타저  마저  추공  아이템 
 800   0 650 225 225 155   0['현무투', '현무피갑', '현무의손목보호구', '현무의요대', '현무의전투화']
 925   0 600 200 225 150   0['황제의머리띠', '현무피갑', '현무의손목보호구', '황제의요대', '현무의전투화']
 825   0 600 250 225 150   0['황제의머리띠', '현무피갑', '현무의손목보호구', '치우천왕요대', '현무의전투화']
 825   0 500 250 225 150   0['황제의머리띠', '현무피갑', '현무의손목보호구', '파멸자의요대', '현무의전투화']
 900   0 375 300 225 150   0['치우천왕투', '황제의갑옷', '파멸자의장갑', '파멸자의요대', '파멸자의전투화']
1000  50 425 150 225 150   0['백호의머리띠', '치우천왕갑', '백호손목보호대', '황제의요대', '백호의전투화']
 950  50 475 200 225 150   0['백호의머리띠', '치우천왕갑', '백호손목보호대', '치우천왕요대', '백호의전투화']
 950  50 325 200 225 150   0['백호의머리띠', '치우천왕갑', '백호손목보호대', '파멸자의요대', '백호의전투화']
1125   0 275 125 220 160   0['백호의머리띠', '백호피갑', '백호손목보호대', '백호의요대', '백호의전투화']
 900  50 375 300 220 155   0['황제의머리띠', '치우천왕갑', '파멸자의장갑', '파멸자의요대', '파멸자의전투화']
1180  30 590 140 220 150   0['황제의머리띠', '황제의갑옷', '황제의장갑', '담덕태황의요대', '황제의신발']
1030  30 615 140 220 150   0['황제의머리띠', '황제의갑옷', '황제의장

 585 160 455 305 200 155   0['파멸자투구', '개마갑주', '치우천왕수', '담덕태황의요대', '담덕태황의신발']
 660 160 405 230 200 155   0['파멸자투구', '개마갑주', '백호손목보호대', '담덕태황의요대', '담덕태황의신발']
 705 180 565 265 200 155  30['고구려태황투구', '치우천왕갑', '파멸자의장갑', '담덕태황의요대', '치우천왕군화']
 730 180 465 290 200 155  30['고구려태황투구', '치우천왕갑', '파멸자의장갑', '담덕태황의요대', '파멸자의전투화']
 680 130 565 315 200 155  30['고구려태황투구', '현무피갑', '치우천왕수', '담덕태황의요대', '치우천왕군화']
 705 130 465 290 200 155  30['고구려태황투구', '현무피갑', '치우천왕수', '담덕태황의요대', '파멸자의전투화']
 755 130 465 240 200 155  30['고구려태황투구', '현무피갑', '백호손목보호대', '담덕태황의요대', '치우천왕군화']
 780 130 415 215 200 155  30['고구려태황투구', '현무피갑', '백호손목보호대', '담덕태황의요대', '파멸자의전투화']
 780 130 490 215 200 155  30['고구려태황투구', '백호피갑', '현무의손목보호구', '담덕태황의요대', '치우천왕군화']
 805 130 440 190 200 155  30['고구려태황투구', '백호피갑', '현무의손목보호구', '담덕태황의요대', '파멸자의전투화']
 530 130 615 390 200 155  30['고구려태황투구', '개마갑주', '치우천왕수', '담덕태황의요대', '치우천왕군화']
 555 130 515 365 200 155  30['고구려태황투구', '개마갑주', '치우천왕수', '담덕태황의요대', '파멸자의전투화']
 605 130 515 315 200 155  30['고구려태황투구', '개마갑주

 710 110 630 155 185 170   0['치우천왕투', '현무피갑', '담덕태황의장갑', '담덕태황의요대', '현무의전투화']
 760 110 505 130 185 170   0['치우천왕투', '현무피갑', '담덕태황의장갑', '담덕태황의요대', '백호의전투화']
 635 110 605 230 185 170   0['치우천왕투', '개마갑주', '담덕태황의장갑', '담덕태황의요대', '황제의신발']
 560 110 630 230 185 170   0['치우천왕투', '개마갑주', '담덕태황의장갑', '담덕태황의요대', '현무의전투화']
 610 110 555 205 185 170   0['치우천왕투', '개마갑주', '담덕태황의장갑', '담덕태황의요대', '백호의전투화']
 755 130 490 190 185 170  30['치우천왕투', '고구려태황갑옷', '파멸자의장갑', '담덕태황의요대', '황제의신발']
 680 130 515 190 185 170  30['치우천왕투', '고구려태황갑옷', '파멸자의장갑', '담덕태황의요대', '현무의전투화']
 730 130 440 165 185 170  30['치우천왕투', '고구려태황갑옷', '파멸자의장갑', '담덕태황의요대', '백호의전투화']
 785 160 605 155 185 170   0['현무투', '치우천왕갑', '담덕태황의장갑', '담덕태황의요대', '황제의신발']
 710 160 680 155 185 170   0['현무투', '치우천왕갑', '담덕태황의장갑', '담덕태황의요대', '현무의전투화']
 760 160 555 130 185 170   0['현무투', '치우천왕갑', '담덕태황의장갑', '담덕태황의요대', '백호의전투화']
 855  30 490 190 185 170   0['현무투', '백호피갑', '파멸자의장갑', '담덕태황의요대', '치우천왕군화']
 880  30 440 215 185 170   0['현무투', '백호피갑', '파멸자의장갑', '담덕태황의요대', '파

 605 130 390 165 160 180  30['악령면', '고구려태황갑옷', '현무의손목보호구', '담덕태황의요대', '파멸자의전투화']
 530  30 515 190 160 175   0['현무투', '해왕갑옷', '현무의손목보호구', '담덕태황의요대', '치우천왕군화']
 555  30 465 165 160 175   0['현무투', '해왕갑옷', '현무의손목보호구', '담덕태황의요대', '파멸자의전투화']
 580  30 415 240 160 175   0['백호의머리띠', '해왕갑옷', '치우천왕수', '담덕태황의요대', '치우천왕군화']
 605  30 315 215 160 175   0['백호의머리띠', '해왕갑옷', '치우천왕수', '담덕태황의요대', '파멸자의전투화']
 705  30 315 165 160 175   0['백호의머리띠', '해왕갑옷', '백호손목보호대', '담덕태황의요대', '치우천왕군화']
 730  30 265 140 160 175   0['백호의머리띠', '해왕갑옷', '백호손목보호대', '담덕태황의요대', '파멸자의전투화']
 480  30 365 315 160 175   0['고대금관', '해왕갑옷', '현무의손목보호구', '담덕태황의요대', '치우천왕군화']
 505  30 315 290 160 175   0['고대금관', '해왕갑옷', '현무의손목보호구', '담덕태황의요대', '파멸자의전투화']
 535 110 480 155 160 170   0['치우천왕투', '해왕갑옷', '담덕태황의장갑', '담덕태황의요대', '치우천왕군화']
 560 110 380 130 160 170   0['치우천왕투', '해왕갑옷', '담덕태황의장갑', '담덕태황의요대', '파멸자의전투화']
 605  30 315 215 160 170   0['백호의머리띠', '해왕갑옷', '파멸자의장갑', '담덕태황의요대', '치우천왕군화']
 630  30 265 240 160 170   0['백호의머리띠', '해왕갑옷', '파멸자의장갑', '